In [ ]:
def Teorema_Chino_Resto(KList,N):
    '''
    Función que resuelve un sistema de congruencias implementando el Teorema Chino del Resto.
    Entrada: una lista de números enteros y módulos que forman un sistema de congruencias y un número N que será el 
    módulo de la solución del sistema de congruencias.
    Salida: un número k solución del sistema de congruencias en módulo N.
    '''
    k = 0
    for i in KList:
        m = N/i[1]
        b = inverse_mod(m%i[1],i[1])
        k = k + i[0]*m*b
    k = k%N
    return k

In [ ]:
def Pohlig_Hellman(P,Q):
    '''
    Función que resuelve el problema del logaritmo discreto en una curva elíptica usando el algoritmo Pohlig-Hellman.
    Entrada: dos puntos P y Q de una curva elíptica definida en un cuerpo finito.
    Salida: una lista de números enteros y módulos que forman un sistema de congruencias y un número N que será el 
    módulo de la solución del sistema de congruencias.
    '''
    N = P.order()
    factores = factor(N) #Descomposición de N en potencias de números primos
    KList = [] #Lista donde se almacenan los distintos valores para k
    
    for i in factores:
        T = []
        KiList = [] #Lista donde se almacenan los Ki para obtener k
        Qi = Q
        
        for j in range(i[0]):
            A = int(j*N/i[0])*P
            T.append(A)
        
        for l in range(i[1]):
            B = int(N/i[0]^(l+1))*Qi
            Ki = T.index(B)
            KiList.append(Ki)
            if l < (i[1]-1):
                Qi = Qi - Ki*i[0]^l*P
        k = 0
        
        #Sacamos cuanto vale k en base de la potencia de un factor
        for m in range(i[1]):
            k = k + KiList[m]*i[0]^m
        k = k%i[0]^(i[1])
        KList.append([k,i[0]^(i[1])])
    return Teorema_Chino_Resto(KList,N)

In [ ]:
import numpy as np
i = next_prime(1000)
cuerpos = []
ordenes = []
tiempos_medias = []
tiempos_desviaciones = []
while i < 20000:
    E = EllipticCurve(GF(i),[2,1])
    N = E.order() - 1
    j = 0
    times = []
    while j < 10:
        P = E[randint(1,N)]
        n = randint(1,N)
        Q = n*P
        time_Inicial = cputime()
        k = Pohlig_Hellman(P,Q)
        time_Final = cputime() - time_Inicial
        times.append(time_Final)
        j = j + 1
    tiempos_medias.append(np.array(times).mean())
    tiempos_desviaciones.append(np.array(times).std())
    ordenes.append(N)
    cuerpos.append(i)
    i = next_prime(i)
import csv
with open('Datos_Pohlig_Hellman.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=';')
    k = 0
    while k < len(ordenes):
        writer.writerow([str(cuerpos[k])]+[str(ordenes[k])]+[str(tiempos_medias[k])]+[str(tiempos_desviaciones[k])])
        k = k + 1

In [ ]:
import csv
Datos1 = []
Datos2 = []
with open('Datos_Pohlig_Hellman.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';')
    for row in spamreader:
        Datos1.append(int(row[0]))
        Datos2.append(float(row[2]))
recta = line(zip(Datos1,Datos2),axes_labels=['$Orden$ $Cuerpo$ $Finito$','$Tiempo$ $Medio$ $(s)$'],frame=True)
recta.show()

In [ ]:
import csv
Datos1 = []
Datos2 = []
with open('Datos_Pohlig_Hellman.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';')
    for row in spamreader:
        Datos1.append(int(row[0]))
        Datos2.append(float(row[3]))
recta = line(zip(Datos1,Datos2),axes_labels=['$Orden$ $Cuerpo$ $Finito$','$Desviación$'],frame=True)
recta.show()